#### Llamando a las librerías que nos hagan falta

In [1]:
from conn import GenericConn
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time

#### Generando conexión

In [2]:
conn = GenericConn()

#### Obteniendo datos

In [3]:
q_ciudades = 'SELECT * FROM Ciudades'
q_paises = 'SELECT * FROM Paises'
q_ordenes = 'SELECT * FROM Ordenes'
q_clientes = 'SELECT * FROM Clientes'
q_productos = 'SELECT * FROM Productos'
df_ordenes = conn.get_query(q_ordenes)
df_clientes = conn.get_query(q_clientes)
df_productos = conn.get_query(q_productos)
df_paises = conn.get_query(q_paises)
df_ciudades = conn.get_query(q_ciudades)

#### Elementos básicos de pandas

In [77]:
# df_paises

# Ver resumen del dataframe
df_paises.describe()

## Seleccionando algunas columnas
# df_paises[['Codigo','Nombre']]

## Seleccionando filas
# df_paises[df_paises.index > 237]
# df_paises[df_paises['Nombre'].str.contains('Pe')]
# df_paises[df_paises['Continente'].str.strip()=='North America']


,Codigo,Nombre,NombreLocal,Iso,Iso3,Continente,Region
count,239,239,239,239,239,239,239
unique,239,239,239,239,239,7,25
top,ABW,Aruba ...,Aruba,AW,ABW,Africa,Caribbean
freq,1,1,1,1,1,58,24


#### Quitar duplicados

In [78]:
df_ciudades.columns

Index(['Id', 'Nombre', 'CodigoPais', 'Distrito'], dtype='object')

In [33]:
df_paises_distinct = df_ciudades['CodigoPais']
df_paises_distinct = df_paises_distinct.drop_duplicates()
df_paises_distinct

0       AFG
4       NLD
32      ANT
33      ALB
34      DZA
       ... 
4113    BVT
4172    HMD
4179    IOT
4266    SGS
4300    UMI
Name: CodigoPais, Length: 239, dtype: object

#### Funciones de agregación

In [103]:
df_ordenes['Fecha'] = pd.to_datetime(df_ordenes['Fecha'])
df_ordenes['Periodo'] = df_ordenes['Fecha'].dt.strftime('%Y-%m')
df_ordenes_resumen = df_ordenes.groupby('Periodo').agg(N_Unidades=('Cantidad','sum'), N_Ordenes=('Id','count'))
df_ordenes_resumen.sort_values(['N_Unidades'],ascending=[False])
df_ordenes_resumen.head(10)

,N_Unidades,N_Ordenes
Periodo,,
2010-01,45,1
2010-02,331,4
2010-03,96,1
2010-04,97,2
2010-05,199,3
2010-06,257,3
2010-07,161,2
2010-08,148,3
2010-09,66,1


In [ ]:
'''
with t_base  as
(
    select Fecha,
           Cantidad,
           Id,
           Year(Fecha)*100+MONTH(Fecha) as Periodo
    from Ordenes
)
select top 10 Periodo,
       sum(Cantidad) as N_Unidades,
       count(Id) as N_Ordenes
from t_base
group by Periodo
order by 2 desc
'''

#### Generación de un reporte en Excel

In [36]:
q_ventas_resumen = '''
    with t_base as 
    (
        select Year(Fecha)*100+MONTH(Fecha) as Periodo, 
            o.Id,
            p.Precio as total_venta,
            c.Nombre + ' ' + Apellido as NombresCompletos
        from Ordenes o 
            left join Productos p on o.ProductoId =p.Id 
            left join Clientes c on o.ClienteId=c.Id
    )
    select NombresCompletos,
        Periodo,
        sum(total_venta) as Total_Ventas
    from t_base 
    group by NombresCompletos,Periodo
'''
df_ventas_resumen = conn.get_query(q_ventas_resumen)
df_ventas_resumen

,NombresCompletos,Periodo,Total_Ventas
0,Erica Padilla,201001,14.00
1,Amethyst Mullins,201002,18.00
2,Hu Cain,201002,38.00
3,Isabella Bond,201002,19.00
4,Mary Mcguire,201002,53.00
...,...,...,...
231,Leo Lewis,201911,10.00
232,Leroy Schultz,201911,21.35
233,Chase Lancaster,201912,38.00
234,Dara Osborne,201912,19.00


In [55]:
df_ventas_resumen_pivot = df_ventas_resumen[(df_ventas_resumen['Periodo'] >= 201901) & (df_ventas_resumen['Periodo'] <= 201912)]
df_ventas_resumen_pivot = df_ventas_resumen_pivot.pivot_table(index='NombresCompletos', columns='Periodo', values='Total_Ventas', aggfunc='sum')
df_ventas_resumen_pivot = df_ventas_resumen_pivot.fillna(0)
df_ventas_resumen_pivot = df_ventas_resumen_pivot.reset_index()
df_ventas_resumen_pivot

Periodo,NombresCompletos,201901,201904,201906,201908,201909,201910,201911,201912
0,Amos Cross,0,0,0,0,0,9.20,0,0
1,Astra Walton,0,43.90,0,0,0,0,0,0
2,Brandon Cooper,0,0,0,0,263.50,0,0,0
3,Caleb Marsh,0,0,0,17.00,0,0,0,0
4,Carla Trevino,0,0,0,2.50,0,0,0,0
5,Chase Lancaster,0,0,0,0,0,0,0,38.00
6,Chloe Bird,0,0,0,0,0,16.25,0,0
7,Cyrus Kane,0,0,31.23,0,0,0,0,0
8,Dara Osborne,0,0,0,0,0,0,0,19.00
9,Harrison Newman,0,0,32.00,0,0,0,0,0


In [51]:
df_ventas_resumen_pivot.to_excel('ventas_resumen.xlsx',index=False)

#### ¿Por qué utilizar python?

##### Scrapeo

In [115]:
url = 'https://realpython.github.io/fake-jobs/'
response = requests.get(url)


if response.status_code == 200:
    results = BeautifulSoup(response.content, 'html.parser')
    job_elements = results.find_all("div", class_="card-content")

    arr_jobs = []
    for job_element in job_elements:
        title_element = job_element.find("h2", class_="title")
        company_element = job_element.find("h3", class_="company")
        location_element = job_element.find("p", class_="location")
        
        title = title_element.text.strip()
        company = company_element.text.strip()
        location = location_element.text.strip()
        arr_jobs.append([title,company,location])
        
    df_jobs = pd.DataFrame(arr_jobs, columns=['Title','Company','Location'])

df_jobs

,Title,Company,Location
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA"
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA"
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA"
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP"
4,Product manager,Ramirez Inc,"North Jamieview, AP"
...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE"
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP"
97,Database administrator,Yates-Ferguson,"Port Susan, AE"
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA"


##### Multiprocessing


In [4]:
from multiprocessing import Pool

In [5]:
df_productos.columns

Index(['Id', 'Nombre', 'ProveedorId', 'Costo', 'Precio', 'CategoriaId'], dtype='object')

In [6]:
def crear_txt(nombre_producto):
    ruta = f'outputs/{nombre_producto}.txt'
    with open(ruta,"w") as file:
        file.write(nombre_producto)
        time.sleep(2)

In [7]:
arr_productos = []
for i,r in df_productos.iterrows():
    nombre_producto = r['Nombre']
    # crear_txt(nombre_producto)
    arr_productos.append(nombre_producto)
    
# arr_productos
pool = Pool(8)
pool.map(crear_txt, arr_productos)
pool.close()
pool.join()

